In [2]:
# Import necessary libraries
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import re
import praw
import nltk

In [3]:
# Initialize PRAW (Python Reddit API Wrapper)
user_agent = "StockMarket Sentiment Analysis"
reddit = praw.Reddit(
    client_id="SDD6BO5WvswKQ2DOZrioiQ",
    client_secret="BoGTdq9xg1Wu4kjy3zj_WBLbTInQvQ",
    user_agent=user_agent
)

# Initialize the subreddit
subreddit = reddit.subreddit("wallstreetbets")

In [4]:
# List to hold data for each post
posts = []

# Fetch posts from the subreddit
for submission in subreddit.new(limit=1000):
    post_date = datetime.datetime.fromtimestamp(submission.created_utc)
    posts.append({
        'id': submission.id,
        'title': submission.title,
        'selftext': submission.selftext,
        'score': submission.score,
        'num_comments': submission.num_comments,
        'upvote_ratio': submission.upvote_ratio,       
        'created_date': post_date,
        'url': submission.url
    })

# Convert the list to a DataFrame
raw_df = pd.DataFrame(posts)
raw_df

,id,title,selftext,score,num_comments,upvote_ratio,created_date,url
0,1jqtyta,Wtf you say this now??,He does this every time there’s a crash. Looon...,2,1,1.00,2025-04-03 22:54:57,https://i.redd.it/32pvaan2oose1.jpeg
1,1jqty5e,Trump’s 2015 Prediction Comes True!,"Back in 2015, Trump said if the Dow dropped 10...",3,1,1.00,2025-04-03 22:54:14,https://i.redd.it/pi1om01vnose1.png
2,1jqty1o,Happy Recession Day!!,https://preview.redd.it/twoo1mtrnose1.png?widt...,2,1,1.00,2025-04-03 22:54:07,https://www.reddit.com/r/wallstreetbets/commen...
3,1jqtvum,2 trillion btw,,2,2,1.00,2025-04-03 22:51:38,https://i.redd.it/hzvc9a7hnose1.jpeg
4,1jqtp5i,"Thank you for the operation, Dr. President",,2,1,0.67,2025-04-03 22:44:18,https://i.redd.it/hdd4hecxlose1.jpeg
...,...,...,...,...,...,...,...,...
790,1j4aiup,Time in the Market vs. Timing the Market - 98 ...,,55,45,0.82,2025-03-05 19:47:40,https://www.reddit.com/gallery/1j4aiup
791,1j4acdo,🔥 CAPR DD: The Little Biotech That Could (Or C...,**Ticker**: CAPR (Capricor Therapeutics)\n\n**...,79,129,0.87,2025-03-05 19:40:28,https://www.reddit.com/r/wallstreetbets/commen...
792,1j49sif,This time will be different!,Almost lost it already but some random drop sa...,92,26,0.94,2025-03-05 19:18:36,https://www.reddit.com/gallery/1j49sif
793,1j499lh,MSTR 15k ALL IN YOLO,I WANT 500k GO BACK TO ATH PLEASE. I JUST SAW ...,427,316,0.90,2025-03-05 18:58:04,https://www.reddit.com/gallery/1j499lh


In [5]:
# URL with the S&P 500 companies list on Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Read the HTML table from the page
tables = pd.read_html(url)
sp500_df = tables[0]

# Extract the ticker symbols (the column is usually named "Symbol")
tickers = sp500_df['Symbol'].tolist()

# Clean the ticker symbols to ensure they are in the correct format
tickers_dollar = [f"${ticker.upper()}" for ticker in tickers]
tickers_dollar


['$MMM',
 '$AOS',
 '$ABT',
 '$ABBV',
 '$ACN',
 '$ADBE',
 '$AMD',
 '$AES',
 '$AFL',
 '$A',
 '$APD',
 '$ABNB',
 '$AKAM',
 '$ALB',
 '$ARE',
 '$ALGN',
 '$ALLE',
 '$LNT',
 '$ALL',
 '$GOOGL',
 '$GOOG',
 '$MO',
 '$AMZN',
 '$AMCR',
 '$AEE',
 '$AEP',
 '$AXP',
 '$AIG',
 '$AMT',
 '$AWK',
 '$AMP',
 '$AME',
 '$AMGN',
 '$APH',
 '$ADI',
 '$ANSS',
 '$AON',
 '$APA',
 '$APO',
 '$AAPL',
 '$AMAT',
 '$APTV',
 '$ACGL',
 '$ADM',
 '$ANET',
 '$AJG',
 '$AIZ',
 '$T',
 '$ATO',
 '$ADSK',
 '$ADP',
 '$AZO',
 '$AVB',
 '$AVY',
 '$AXON',
 '$BKR',
 '$BALL',
 '$BAC',
 '$BAX',
 '$BDX',
 '$BRK.B',
 '$BBY',
 '$TECH',
 '$BIIB',
 '$BLK',
 '$BX',
 '$BK',
 '$BA',
 '$BKNG',
 '$BSX',
 '$BMY',
 '$AVGO',
 '$BR',
 '$BRO',
 '$BF.B',
 '$BLDR',
 '$BG',
 '$BXP',
 '$CHRW',
 '$CDNS',
 '$CZR',
 '$CPT',
 '$CPB',
 '$COF',
 '$CAH',
 '$KMX',
 '$CCL',
 '$CARR',
 '$CAT',
 '$CBOE',
 '$CBRE',
 '$CDW',
 '$COR',
 '$CNC',
 '$CNP',
 '$CF',
 '$CRL',
 '$SCHW',
 '$CHTR',
 '$CVX',
 '$CMG',
 '$CB',
 '$CHD',
 '$CI',
 '$CINF',
 '$CTAS',
 '$CSCO',
 '$C',
 '$C

In [6]:
long_blacklist = {'I', 'ARE',  'ON', 'GO', 'NOW', 'CAN', 'UK', 'SO', 'OR', 'OUT', 'SEE', 'ONE', 'LOVE', 'U', 'STAY', 'HAS', 'BY', 'BIG', 'GOOD', 'RIDE', 'EOD', 'ELON', 'WSB',
            'THE', 'A', 'ROPE', 'YOLO', 'TOS', 'CEO', 'DD', 'IT', 'OPEN', 'ATH', 'PM', 'IRS', 'FOR','DEC', 'BE', 'IMO', 'ALL', 'RH', 'EV', 'TOS', 'CFO', 'CTO','DD', 'BTFD',
            'WSB', 'OK', 'PDT', 'RH', 'KYS', 'FD', 'TYS', 'US', 'USA', 'IT', 'ATH', 'RIP', 'BMW', 'GDP', 'OTM', 'ATM', 'ITM', 'IMO', 'LOL', 'AM', 'BE', 'PR', 'PRAY', 'PT', 
            'FBI', 'SEC', 'GOD', 'NOT', 'POS', 'FOMO', 'TL;DR', 'EDIT', 'STILL', 'WTF', 'RAW', 'PM', 'LMAO','LMFAO', 'ROFL', 'EZ', 'RED', 'BEZOS', 'TICK', 'IS', 'PM', 'LPT',
            'GOAT', 'FL', 'CA', 'IL', 'MACD', 'HQ', 'OP', 'PS', 'AH', 'TL', 'JAN', 'FEB', 'JUL', 'AUG', 'SEP', 'SEPT', 'OCT', 'NOV', 'FDA', 'IV', 'ER', 'IPO', 'MILF', 'BUT', 
            'SSN', 'FIFA', 'USD', 'CPU', 'AT', 'GG'}

blacklist = {'A', 'ARE', 'ALL', 'DD', 'IT', 'HAS', 'ON', 'PM', 'NOW', 'SO'}

In [8]:
# Create a regex pattern from the S&P 500 tickers list, using word boundaries to avoid false matches
pattern = r'\b(?:' + '|'.join(map(re.escape, tickers)) + r')\b'

# Combine title and selftext into one column to search both
raw_df['combined_text'] = raw_df['title'].fillna('') + " " + raw_df['selftext'].fillna('')

# Filter the DataFrame to include only posts that mention any S&P 500 stock ticker
filtered_reddit = raw_df[raw_df['combined_text'].str.contains(pattern, flags=re.IGNORECASE, regex=True)].copy()
filtered_reddit


,id,title,selftext,score,num_comments,upvote_ratio,created_date,url,combined_text
0,1jqtyta,Wtf you say this now??,He does this every time there’s a crash. Looon...,2,1,1.00,2025-04-03 22:54:57,https://i.redd.it/32pvaan2oose1.jpeg,Wtf you say this now?? He does this every time...
1,1jqty5e,Trump’s 2015 Prediction Comes True!,"Back in 2015, Trump said if the Dow dropped 10...",3,1,1.00,2025-04-03 22:54:14,https://i.redd.it/pi1om01vnose1.png,Trump’s 2015 Prediction Comes True! Back in 20...
2,1jqty1o,Happy Recession Day!!,https://preview.redd.it/twoo1mtrnose1.png?widt...,2,1,1.00,2025-04-03 22:54:07,https://www.reddit.com/r/wallstreetbets/commen...,Happy Recession Day!! https://preview.redd.it/...
6,1jqtekc,75k Gain On Puts,"I posted my positions last night, but here’s t...",46,6,1.00,2025-04-03 22:32:49,https://www.reddit.com/gallery/1jqtekc,75k Gain On Puts I posted my positions last ni...
7,1jqtc5a,"Started options last week, nuked half my portf...",I have no idea what I’m doing.\nThis is genuin...,15,13,0.89,2025-04-03 22:30:11,https://www.reddit.com/gallery/1jqtc5a,"Started options last week, nuked half my portf..."
...,...,...,...,...,...,...,...,...,...
788,1j4bf01,European/Asian stocks to pay more attention to...,"Considering continuing tariff, a possible gove...",690,286,0.89,2025-03-05 20:22:56,https://www.reddit.com/r/wallstreetbets/commen...,European/Asian stocks to pay more attention to...
789,1j4alok,The Stagflation Celebration: the case for VIX ...,We are on the precipice of massive volatility ...,309,130,0.93,2025-03-05 19:50:47,https://i.redd.it/r9j577ih3xme1.jpeg,The Stagflation Celebration: the case for VIX ...
791,1j4acdo,🔥 CAPR DD: The Little Biotech That Could (Or C...,**Ticker**: CAPR (Capricor Therapeutics)\n\n**...,79,129,0.87,2025-03-05 19:40:28,https://www.reddit.com/r/wallstreetbets/commen...,🔥 CAPR DD: The Little Biotech That Could (Or C...
792,1j49sif,This time will be different!,Almost lost it already but some random drop sa...,92,26,0.94,2025-03-05 19:18:36,https://www.reddit.com/gallery/1j49sif,This time will be different! Almost lost it al...
